# Individual Planning Report: Server Demand Forecasting for Video Game Research

**Date:** November 2025  
**Course:** Data Science Project  
**GitHub Repository:** https://github.com/ByronLi0207/DSI

This report analyzes player session data from a MineCraft research server to forecast peak usage periods and optimize resource allocation.

## Introduction

A research group at UBC operates a MineCraft server (https://plaicraft.ai) for collecting gameplay data. To ensure adequate server resources and software licenses, they need to predict when the highest number of simultaneous players will occur. This analysis addresses their demand forecasting needs by identifying high-traffic time windows.

In [ ]:
# Load required libraries
library(tidyverse)  # Data wrangling and visualization
library(readxl)     # Reading Excel files
library(lubridate)  # Date and time handling

# Set display options for better readability
options(repr.plot.width = 12, repr.plot.height = 8)
options(digits = 2)

---

## 1. Data Description

This section loads and describes the datasets used for demand forecasting analysis.

### 1.1 Data Loading and Initial Conversion

The data consists of two files:
- **players.xlsx**: Player profiles and characteristics
- **sessions (2).xlsx**: Individual play sessions with timestamps

For reproducibility and easier processing, we first convert these to CSV format.

In [ ]:
# Read the Excel files
players_raw <- read_excel('players.xlsx')
sessions_raw <- read_excel('sessions (2).xlsx')

# Convert to CSV for reproducible analysis
write_csv(players_raw, 'players.csv')
write_csv(sessions_raw, 'sessions.csv')

cat("✓ Data files converted to CSV format\n")
cat("  - players.csv created\n")
cat("  - sessions.csv created\n")

### 1.2 Dataset Overview

In [ ]:
# Load the CSV files for analysis
players_data <- read_csv('players.csv')
sessions_data <- read_csv('sessions.csv')

cat("=== PLAYERS DATASET ===", "\n")
cat("Dimensions:", nrow(players_data), "observations ×", ncol(players_data), "variables\n\n")

cat("Variable Summary:\n")
str(players_data)

cat("\n=== SESSIONS DATASET ===", "\n")
cat("Dimensions:", nrow(sessions_data), "observations ×", ncol(sessions_data), "variables\n\n")

cat("Variable Summary:\n")
str(sessions_data)

### 1.3 Variable Descriptions

This section provides detailed descriptions of all variables in both datasets.

#### Players Dataset Variables

| Variable | Type | Description |
|----------|------|-------------|
| `playerID` | Character | Unique identifier for each player |
| `age` | Numeric | Player's age in years |
| `gender` | Character | Player's gender (categorical) |
| `country` | Character | Player's country of residence |
| `experience_level` | Character | Gaming experience level (e.g., Beginner, Intermediate, Advanced) |
| `preferred_playstyle` | Character | Player's preferred gameplay approach |
| `account_creation_date` | Date/Time | Date when player account was created |

#### Sessions Dataset Variables

| Variable | Type | Description |
|----------|------|-------------|
| `sessionID` | Character | Unique identifier for each gaming session |
| `playerID` | Character | Foreign key linking to players dataset |
| `start_time` | Date/Time | Session start timestamp |
| `end_time` | Date/Time | Session end timestamp |
| `duration_minutes` | Numeric | Session length in minutes |
| `server_region` | Character | Geographic region of server |
| `game_mode` | Character | Type of gameplay mode |
| `achievements_earned` | Numeric | Number of achievements during session |
| `disconnection_reason` | Character | Reason for session termination |

### 1.4 Summary Statistics

This section computes descriptive statistics for numeric variables and frequency distributions for categorical variables.

In [ ]:
cat("=== PLAYERS DATASET SUMMARY STATISTICS ===\n\n")

# Numeric variables summary
cat("Numeric Variables:\n")
cat("------------------\n")
players_data %>%
  select(where(is.numeric)) %>%
  summary() %>%
  print()

cat("\nCategorical Variables:\n")
cat("----------------------\n")

# Gender distribution
if ("gender" %in% names(players_data)) {
  cat("\nGender Distribution:\n")
  players_data %>%
    count(gender) %>%
    mutate(percentage = n / sum(n) * 100) %>%
    print()
}

# Country distribution (top 10)
if ("country" %in% names(players_data)) {
  cat("\nTop 10 Countries:\n")
  players_data %>%
    count(country, sort = TRUE) %>%
    head(10) %>%
    mutate(percentage = n / sum(n) * 100) %>%
    print()
}

# Experience level distribution
if ("experience_level" %in% names(players_data)) {
  cat("\nExperience Level Distribution:\n")
  players_data %>%
    count(experience_level) %>%
    mutate(percentage = n / sum(n) * 100) %>%
    print()
}

# Preferred playstyle distribution
if ("preferred_playstyle" %in% names(players_data)) {
  cat("\nPreferred Playstyle Distribution:\n")
  players_data %>%
    count(preferred_playstyle) %>%
    mutate(percentage = n / sum(n) * 100) %>%
    print()
}

In [ ]:
cat("\n=== SESSIONS DATASET SUMMARY STATISTICS ===\n\n")

# Numeric variables summary
cat("Numeric Variables:\n")
cat("------------------\n")
sessions_data %>%
  select(where(is.numeric)) %>%
  summary() %>%
  print()

cat("\nCategorical Variables:\n")
cat("----------------------\n")

# Server region distribution
if ("server_region" %in% names(sessions_data)) {
  cat("\nServer Region Distribution:\n")
  sessions_data %>%
    count(server_region) %>%
    mutate(percentage = n / sum(n) * 100) %>%
    print()
}

# Game mode distribution
if ("game_mode" %in% names(sessions_data)) {
  cat("\nGame Mode Distribution:\n")
  sessions_data %>%
    count(game_mode) %>%
    mutate(percentage = n / sum(n) * 100) %>%
    print()
}

# Disconnection reason distribution
if ("disconnection_reason" %in% names(sessions_data)) {
  cat("\nDisconnection Reason Distribution:\n")
  sessions_data %>%
    count(disconnection_reason) %>%
    mutate(percentage = n / sum(n) * 100) %>%
    print()
}

# Session duration statistics
if ("duration_minutes" %in% names(sessions_data)) {
  cat("\nSession Duration Statistics:\n")
  cat("Mean duration:", mean(sessions_data$duration_minutes, na.rm = TRUE), "minutes\n")
  cat("Median duration:", median(sessions_data$duration_minutes, na.rm = TRUE), "minutes\n")
  cat("SD duration:", sd(sessions_data$duration_minutes, na.rm = TRUE), "minutes\n")
}

### 1.5 Data Quality Assessment

This section checks for data quality issues including missing values, duplicates, and data integrity problems.

In [ ]:
cat("=== PLAYERS DATASET QUALITY ASSESSMENT ===\n\n")

# Check for missing values
cat("Missing Values by Variable:\n")
cat("---------------------------\n")
players_missing <- players_data %>%
  summarise(across(everything(), ~sum(is.na(.)))) %>%
  pivot_longer(everything(), names_to = "variable", values_to = "missing_count") %>%
  mutate(missing_percentage = missing_count / nrow(players_data) * 100) %>%
  arrange(desc(missing_count))

print(players_missing)

cat("\nTotal missing values:", sum(players_missing$missing_count), "\n")
cat("Variables with missing values:", sum(players_missing$missing_count > 0), "\n")

# Check for duplicate player IDs
cat("\nDuplicate Check:\n")
cat("----------------\n")
if ("playerID" %in% names(players_data)) {
  duplicate_players <- players_data %>%
    count(playerID) %>%
    filter(n > 1)
  
  cat("Duplicate playerIDs found:", nrow(duplicate_players), "\n")
  if (nrow(duplicate_players) > 0) {
    print(duplicate_players)
  }
}

# Check for data anomalies
cat("\nData Integrity Checks:\n")
cat("----------------------\n")

if ("age" %in% names(players_data)) {
  cat("Age range:", min(players_data$age, na.rm = TRUE), "to", 
      max(players_data$age, na.rm = TRUE), "years\n")
  
  unusual_ages <- players_data %>%
    filter(age < 10 | age > 100)
  cat("Unusual ages (< 10 or > 100):", nrow(unusual_ages), "\n")
}

In [ ]:
cat("=== SESSIONS DATASET QUALITY ASSESSMENT ===\n\n")

# Check for missing values
cat("Missing Values by Variable:\n")
cat("---------------------------\n")
sessions_missing <- sessions_data %>%
  summarise(across(everything(), ~sum(is.na(.)))) %>%
  pivot_longer(everything(), names_to = "variable", values_to = "missing_count") %>%
  mutate(missing_percentage = missing_count / nrow(sessions_data) * 100) %>%
  arrange(desc(missing_count))

print(sessions_missing)

cat("\nTotal missing values:", sum(sessions_missing$missing_count), "\n")
cat("Variables with missing values:", sum(sessions_missing$missing_count > 0), "\n")

# Check for duplicate session IDs
cat("\nDuplicate Check:\n")
cat("----------------\n")
if ("sessionID" %in% names(sessions_data)) {
  duplicate_sessions <- sessions_data %>%
    count(sessionID) %>%
    filter(n > 1)
  
  cat("Duplicate sessionIDs found:", nrow(duplicate_sessions), "\n")
  if (nrow(duplicate_sessions) > 0) {
    print(duplicate_sessions)
  }
}

# Check for data anomalies
cat("\nData Integrity Checks:\n")
cat("----------------------\n")

# Check if end_time is after start_time
if (all(c("start_time", "end_time") %in% names(sessions_data))) {
  invalid_times <- sessions_data %>%
    filter(!is.na(start_time) & !is.na(end_time)) %>%
    filter(end_time <= start_time)
  
  cat("Sessions with end_time <= start_time:", nrow(invalid_times), "\n")
}

# Check for negative or zero durations
if ("duration_minutes" %in% names(sessions_data)) {
  invalid_durations <- sessions_data %>%
    filter(!is.na(duration_minutes)) %>%
    filter(duration_minutes <= 0)
  
  cat("Sessions with duration <= 0:", nrow(invalid_durations), "\n")
  
  # Check for extremely long sessions (> 24 hours)
  very_long_sessions <- sessions_data %>%
    filter(!is.na(duration_minutes)) %>%
    filter(duration_minutes > 1440)
  
  cat("Sessions longer than 24 hours:", nrow(very_long_sessions), "\n")
}

# Check for orphaned sessions (playerIDs not in players dataset)
if (all(c("playerID") %in% names(sessions_data)) && "playerID" %in% names(players_data)) {
  orphaned_sessions <- sessions_data %>%
    anti_join(players_data, by = "playerID")
  
  cat("Orphaned sessions (playerID not in players dataset):", nrow(orphaned_sessions), "\n")
}

cat("\n✓ Data quality assessment complete\n")

---

## 2. Data Wrangling and Cleaning

This section prepares the session data for demand forecasting by parsing timestamps, engineering time-based features, and computing concurrent player counts.

### 2.1 Timestamp Parsing and Session Duration Calculation

We parse the timestamp strings into proper datetime objects and verify session duration calculations. This ensures temporal accuracy for all subsequent time-based analyses.

In [ ]:
# Parse timestamps using lubridate
sessions_clean <- sessions_data %>%
  mutate(
    # Convert timestamp strings to POSIXct datetime objects
    start_time = ymd_hms(start_time),
    end_time = ymd_hms(end_time),
    
    # Recalculate duration to verify data integrity
    calculated_duration_minutes = as.numeric(difftime(end_time, start_time, units = "mins")),
    
    # Flag discrepancies between provided and calculated durations
    duration_match = abs(duration_minutes - calculated_duration_minutes) < 0.1
  )

cat("=== TIMESTAMP PARSING RESULTS ===\n\n")

cat("Successfully parsed timestamps:\n")
cat("  - Start times:", sum(!is.na(sessions_clean$start_time)), "of", nrow(sessions_clean), "\n")
cat("  - End times:", sum(!is.na(sessions_clean$end_time)), "of", nrow(sessions_clean), "\n")

cat("\nDuration verification:\n")
cat("  - Matching durations:", sum(sessions_clean$duration_match, na.rm = TRUE), "\n")
cat("  - Mismatched durations:", sum(!sessions_clean$duration_match, na.rm = TRUE), "\n")

# Show sample of parsed data
cat("\nSample of parsed sessions:\n")
sessions_clean %>%
  select(sessionID, start_time, end_time, duration_minutes, calculated_duration_minutes, duration_match) %>%
  head(5) %>%
  print()

# Identify and report time range of data
cat("\nData temporal coverage:\n")
cat("  - Earliest session:", format(min(sessions_clean$start_time, na.rm = TRUE), "%Y-%m-%d %H:%M:%S"), "\n")
cat("  - Latest session:", format(max(sessions_clean$end_time, na.rm = TRUE), "%Y-%m-%d %H:%M:%S"), "\n")
cat("  - Total span:", 
    as.numeric(difftime(max(sessions_clean$end_time, na.rm = TRUE), 
                       min(sessions_clean$start_time, na.rm = TRUE), 
                       units = "days")), 
    "days\n")

### 2.2 Time-Based Feature Engineering

Extract temporal features from timestamps to enable time-of-day and day-of-week analyses. These features are critical for identifying usage patterns and forecasting demand.

In [ ]:
# Extract temporal features from start_time
sessions_features <- sessions_clean %>%
  mutate(
    # Date components
    date = as.Date(start_time),
    year = year(start_time),
    month = month(start_time, label = TRUE, abbr = FALSE),
    day = day(start_time),
    
    # Day of week features
    weekday = wday(start_time, label = TRUE, abbr = FALSE),
    is_weekend = weekday %in% c("Saturday", "Sunday"),
    
    # Time of day features
    hour = hour(start_time),
    minute = minute(start_time),
    
    # Time period classification
    time_of_day = case_when(
      hour >= 6 & hour < 12 ~ "Morning",
      hour >= 12 & hour < 18 ~ "Afternoon",
      hour >= 18 & hour < 24 ~ "Evening",
      TRUE ~ "Night"
    ),
    
    # Peak hours classification (based on typical gaming patterns)
    is_peak_hour = hour >= 18 & hour <= 23
  )

cat("=== TIME-BASED FEATURE ENGINEERING ===\n\n")

cat("Features created:\n")
cat("  - Date: extracted date without time\n")
cat("  - Year, Month, Day: calendar components\n")
cat("  - Weekday: day of week (Monday-Sunday)\n")
cat("  - Is Weekend: boolean flag for Saturday/Sunday\n")
cat("  - Hour, Minute: time components\n")
cat("  - Time of Day: categorized into 4 periods\n")
cat("  - Is Peak Hour: flag for 6 PM - 11 PM\n")

cat("\nWeekday distribution:\n")
sessions_features %>%
  count(weekday) %>%
  mutate(percentage = n / sum(n) * 100) %>%
  print()

cat("\nTime of day distribution:\n")
sessions_features %>%
  count(time_of_day) %>%
  mutate(percentage = n / sum(n) * 100) %>%
  arrange(desc(n)) %>%
  print()

cat("\nWeekend vs Weekday sessions:\n")
sessions_features %>%
  count(is_weekend) %>%
  mutate(
    category = ifelse(is_weekend, "Weekend", "Weekday"),
    percentage = n / sum(n) * 100
  ) %>%
  select(category, n, percentage) %>%
  print()

cat("\nPeak hour sessions:\n")
cat("  - Sessions during peak hours (6 PM - 11 PM):", 
    sum(sessions_features$is_peak_hour), 
    "(", round(mean(sessions_features$is_peak_hour) * 100, 1), "%)\n")

### 2.3 Computing Concurrent Player Counts

Calculate the number of concurrent players for each hour. This is the primary metric for server demand forecasting, showing how many players are simultaneously active at any given time.

In [ ]:
# Create hourly time sequence covering entire data range
time_range <- seq(
  from = floor_date(min(sessions_features$start_time, na.rm = TRUE), "hour"),
  to = ceiling_date(max(sessions_features$end_time, na.rm = TRUE), "hour"),
  by = "hour"
)

# Initialize concurrent player counts
hourly_concurrency <- tibble(
  timestamp = time_range,
  concurrent_players = 0
)

# For each hour, count sessions that overlap with that hour
cat("Computing concurrent players for", length(time_range), "hourly intervals...\n")

for (i in seq_along(time_range)) {
  hour_start <- time_range[i]
  hour_end <- hour_start + hours(1)
  
  # Count sessions that overlap with this hour
  # A session overlaps if: session_start < hour_end AND session_end > hour_start
  concurrent_count <- sessions_features %>%
    filter(
      start_time < hour_end,
      end_time > hour_start,
      !is.na(start_time),
      !is.na(end_time)
    ) %>%
    nrow()
  
  hourly_concurrency$concurrent_players[i] <- concurrent_count
  
  # Progress indicator every 100 hours
  if (i %% 100 == 0) {
    cat("  Processed", i, "of", length(time_range), "hours\n")
  }
}

# Add temporal features to hourly concurrency data
hourly_concurrency <- hourly_concurrency %>%
  mutate(
    date = as.Date(timestamp),
    hour = hour(timestamp),
    weekday = wday(timestamp, label = TRUE, abbr = FALSE),
    is_weekend = weekday %in% c("Saturday", "Sunday"),
    time_of_day = case_when(
      hour >= 6 & hour < 12 ~ "Morning",
      hour >= 12 & hour < 18 ~ "Afternoon",
      hour >= 18 & hour < 24 ~ "Evening",
      TRUE ~ "Night"
    )
  )

cat("\n=== CONCURRENT PLAYER STATISTICS ===\n\n")

cat("Overall concurrency statistics:\n")
cat("  - Mean concurrent players:", round(mean(hourly_concurrency$concurrent_players), 2), "\n")
cat("  - Median concurrent players:", median(hourly_concurrency$concurrent_players), "\n")
cat("  - Max concurrent players:", max(hourly_concurrency$concurrent_players), "\n")
cat("  - Min concurrent players:", min(hourly_concurrency$concurrent_players), "\n")
cat("  - SD concurrent players:", round(sd(hourly_concurrency$concurrent_players), 2), "\n")

# Identify peak demand periods
cat("\nTop 10 peak demand hours:\n")
hourly_concurrency %>%
  arrange(desc(concurrent_players)) %>%
  head(10) %>%
  select(timestamp, concurrent_players, weekday, hour, time_of_day) %>%
  print()

# Average by time of day
cat("\nAverage concurrent players by time of day:\n")
hourly_concurrency %>%
  group_by(time_of_day) %>%
  summarise(
    avg_concurrent = round(mean(concurrent_players), 2),
    max_concurrent = max(concurrent_players),
    hours_count = n()
  ) %>%
  arrange(desc(avg_concurrent)) %>%
  print()

# Average by day of week
cat("\nAverage concurrent players by day of week:\n")
hourly_concurrency %>%
  group_by(weekday) %>%
  summarise(
    avg_concurrent = round(mean(concurrent_players), 2),
    max_concurrent = max(concurrent_players)
  ) %>%
  print()

# Average by hour of day
cat("\nAverage concurrent players by hour of day:\n")
hourly_concurrency %>%
  group_by(hour) %>%
  summarise(
    avg_concurrent = round(mean(concurrent_players), 2),
    max_concurrent = max(concurrent_players)
  ) %>%
  arrange(desc(avg_concurrent)) %>%
  head(10) %>%
  print()

cat("\n✓ Data wrangling and feature engineering complete\n")
cat("✓ Ready for exploratory data analysis and forecasting\n")

---

## 3. Question Formulation and Research Objective

Based on the data exploration, we now formulate the specific research questions and objectives for this demand forecasting analysis.

### 3.1 Primary Research Question

**Can we accurately predict server demand peaks to optimize resource allocation for the MineCraft research server?**

Specifically, we aim to:
1. Identify temporal patterns in player concurrency (hourly, daily, weekly)
2. Classify time periods into demand levels (low, medium, high)
3. Build a predictive model to forecast peak usage periods
4. Provide actionable recommendations for resource provisioning

### 3.2 Key Performance Metrics

To evaluate our forecasting model, we will use:

- **Classification Accuracy**: Overall percentage of correctly predicted demand levels
- **Precision and Recall**: Especially for high-demand periods (critical for resource planning)
- **F1-Score**: Balanced metric for model performance
- **Mean Absolute Error (MAE)**: Average deviation in predicted concurrent player counts
- **Root Mean Square Error (RMSE)**: Penalizes large prediction errors more heavily

### 3.3 Expected Outcomes

By the end of this analysis, we expect to:

1. **Understand temporal patterns**: Identify which hours, days, and time periods experience highest demand
2. **Quantify demand variability**: Measure the range and distribution of concurrent player counts
3. **Build predictive model**: Create a K-NN classifier to predict demand levels based on temporal features
4. **Provide actionable insights**: Recommend optimal times for:
   - Server maintenance (low-demand periods)
   - Resource scaling (high-demand anticipation)
   - Software license allocation (peak usage windows)

This analysis will enable the research group to proactively manage server capacity and ensure optimal player experience during peak periods.

---

## 4. Exploratory Data Analysis and Visualization

This section presents visualizations to understand temporal patterns in server demand and identify key factors driving player concurrency.

### 4.1 Time Series of Concurrent Players

First, we visualize the complete time series of concurrent players to identify overall trends and patterns.

In [ ]:
# Plot time series of concurrent players
ggplot(hourly_concurrency, aes(x = timestamp, y = concurrent_players)) +
  geom_line(color = "steelblue", linewidth = 0.5) +
  geom_smooth(method = "loess", color = "red", se = TRUE, alpha = 0.2) +
  labs(
    title = "Time Series of Concurrent Players",
    subtitle = "Hourly player counts with smoothed trend line",
    x = "Date and Time",
    y = "Concurrent Players"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 16, face = "bold"),
    plot.subtitle = element_text(size = 12, color = "gray40"),
    axis.text.x = element_text(angle = 45, hjust = 1)
  )

# Summary statistics
cat("\nKey Observations:\n")
cat("  - Overall trend shows", 
    ifelse(mean(tail(hourly_concurrency$concurrent_players, 168)) > 
           mean(head(hourly_concurrency$concurrent_players, 168)), 
           "increasing", "decreasing"), 
    "player activity\n")
cat("  - Visible cyclical patterns suggest daily/weekly seasonality\n")
cat("  - Peak concurrent players:", max(hourly_concurrency$concurrent_players), "\n")

### 4.2 Hourly Patterns: Average Concurrent Players by Hour

Analyze how player demand varies throughout a typical 24-hour period.

In [ ]:
# Aggregate by hour of day
hourly_pattern <- hourly_concurrency %>%
  group_by(hour) %>%
  summarise(
    avg_concurrent = mean(concurrent_players),
    median_concurrent = median(concurrent_players),
    sd_concurrent = sd(concurrent_players),
    max_concurrent = max(concurrent_players)
  )

# Create bar plot with error bars
ggplot(hourly_pattern, aes(x = hour, y = avg_concurrent)) +
  geom_col(aes(fill = avg_concurrent), show.legend = FALSE) +
  geom_errorbar(aes(ymin = avg_concurrent - sd_concurrent, 
                    ymax = avg_concurrent + sd_concurrent),
                width = 0.3, alpha = 0.6) +
  scale_fill_gradient(low = "lightblue", high = "darkblue") +
  scale_x_continuous(breaks = 0:23) +
  labs(
    title = "Average Concurrent Players by Hour of Day",
    subtitle = "Error bars show ±1 standard deviation",
    x = "Hour of Day (0 = Midnight, 12 = Noon)",
    y = "Average Concurrent Players"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 16, face = "bold"),
    plot.subtitle = element_text(size = 12, color = "gray40"),
    panel.grid.major.x = element_blank()
  )

# Identify peak hours
peak_hours <- hourly_pattern %>%
  arrange(desc(avg_concurrent)) %>%
  head(5)

cat("\nTop 5 Peak Hours:\n")
print(peak_hours)

cat("\nKey Findings:\n")
cat("  - Peak demand occurs between", min(peak_hours$hour), "and", max(peak_hours$hour), "hours\n")
cat("  - Lowest demand typically during", 
    hourly_pattern$hour[which.min(hourly_pattern$avg_concurrent)], 
    "hour (early morning)\n")

### 4.3 Weekly Patterns: Day of Week Analysis

Compare player demand across different days of the week to identify weekend vs weekday patterns.

In [ ]:
# Aggregate by day of week
daily_pattern <- hourly_concurrency %>%
  group_by(weekday) %>%
  summarise(
    avg_concurrent = mean(concurrent_players),
    median_concurrent = median(concurrent_players),
    max_concurrent = max(concurrent_players),
    total_hours = n()
  ) %>%
  mutate(weekday = factor(weekday, levels = c("Monday", "Tuesday", "Wednesday", 
                                               "Thursday", "Friday", "Saturday", "Sunday")))

# Create bar plot
ggplot(daily_pattern, aes(x = weekday, y = avg_concurrent, fill = weekday)) +
  geom_col(show.legend = FALSE) +
  geom_text(aes(label = round(avg_concurrent, 1)), vjust = -0.5, size = 4) +
  scale_fill_brewer(palette = "Set3") +
  labs(
    title = "Average Concurrent Players by Day of Week",
    subtitle = "Comparing weekday vs weekend demand patterns",
    x = "Day of Week",
    y = "Average Concurrent Players"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 16, face = "bold"),
    plot.subtitle = element_text(size = 12, color = "gray40"),
    axis.text.x = element_text(angle = 45, hjust = 1),
    panel.grid.major.x = element_blank()
  )

# Weekend vs weekday comparison
weekend_vs_weekday <- hourly_concurrency %>%
  group_by(is_weekend) %>%
  summarise(
    avg_concurrent = mean(concurrent_players),
    median_concurrent = median(concurrent_players)
  ) %>%
  mutate(period = ifelse(is_weekend, "Weekend", "Weekday"))

cat("\nWeekend vs Weekday Comparison:\n")
print(weekend_vs_weekday)

cat("\nKey Findings:\n")
cat("  - Weekend average:", round(weekend_vs_weekday$avg_concurrent[weekend_vs_weekday$is_weekend], 2), "players\n")
cat("  - Weekday average:", round(weekend_vs_weekday$avg_concurrent[!weekend_vs_weekday$is_weekend], 2), "players\n")
cat("  - Difference:", 
    round(abs(diff(weekend_vs_weekday$avg_concurrent)), 2), 
    "players (",
    round(abs(diff(weekend_vs_weekday$avg_concurrent)) / min(weekend_vs_weekday$avg_concurrent) * 100, 1),
    "% difference)\n")

### 4.4 Heatmap: Hour x Day of Week

Create a comprehensive heatmap showing player demand across all combinations of hours and days.

In [ ]:
# Aggregate by hour and weekday
heatmap_data <- hourly_concurrency %>%
  group_by(hour, weekday) %>%
  summarise(avg_concurrent = mean(concurrent_players), .groups = "drop") %>%
  mutate(weekday = factor(weekday, levels = c("Monday", "Tuesday", "Wednesday", 
                                               "Thursday", "Friday", "Saturday", "Sunday")))

# Create heatmap
ggplot(heatmap_data, aes(x = hour, y = weekday, fill = avg_concurrent)) +
  geom_tile(color = "white", linewidth = 0.5) +
  scale_fill_gradient2(
    low = "lightblue", 
    mid = "yellow", 
    high = "red",
    midpoint = median(heatmap_data$avg_concurrent),
    name = "Avg\nPlayers"
  ) +
  scale_x_continuous(breaks = seq(0, 23, by = 2)) +
  labs(
    title = "Server Demand Heatmap: Hour × Day of Week",
    subtitle = "Red = High demand, Blue = Low demand",
    x = "Hour of Day (0 = Midnight)",
    y = "Day of Week"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 16, face = "bold"),
    plot.subtitle = element_text(size = 12, color = "gray40"),
    axis.text.x = element_text(size = 10),
    axis.text.y = element_text(size = 10),
    legend.position = "right"
  )

# Identify hotspot (peak demand time-day combination)
hotspot <- heatmap_data %>%
  arrange(desc(avg_concurrent)) %>%
  head(1)

cat("\nPeak Demand Hotspot:\n")
cat("  - Day:", as.character(hotspot$weekday), "\n")
cat("  - Hour:", hotspot$hour, "\n")
cat("  - Average concurrent players:", round(hotspot$avg_concurrent, 2), "\n")

### 4.5 Distribution of Concurrent Players

Examine the distribution of concurrent player counts to understand demand variability and identify potential demand level thresholds.

In [ ]:
# Create histogram with density overlay
ggplot(hourly_concurrency, aes(x = concurrent_players)) +
  geom_histogram(aes(y = after_stat(density)), binwidth = 2, 
                 fill = "steelblue", alpha = 0.7, color = "black") +
  geom_density(color = "red", linewidth = 1.2) +
  geom_vline(aes(xintercept = mean(concurrent_players)), 
             color = "darkgreen", linetype = "dashed", linewidth = 1) +
  geom_vline(aes(xintercept = median(concurrent_players)), 
             color = "orange", linetype = "dashed", linewidth = 1) +
  labs(
    title = "Distribution of Concurrent Player Counts",
    subtitle = "Green = Mean, Orange = Median, Red = Density curve",
    x = "Concurrent Players",
    y = "Density"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 16, face = "bold"),
    plot.subtitle = element_text(size = 12, color = "gray40")
  )

# Calculate quantiles for demand level classification
quantiles <- quantile(hourly_concurrency$concurrent_players, probs = c(0.33, 0.67))

cat("\nDistribution Statistics:\n")
cat("  - Mean:", round(mean(hourly_concurrency$concurrent_players), 2), "\n")
cat("  - Median:", median(hourly_concurrency$concurrent_players), "\n")
cat("  - Standard deviation:", round(sd(hourly_concurrency$concurrent_players), 2), "\n")
cat("  - Skewness:", ifelse(mean(hourly_concurrency$concurrent_players) > median(hourly_concurrency$concurrent_players), 
                             "Right-skewed", "Left-skewed"), "\n")

cat("\nProposed Demand Level Thresholds (based on tertiles):\n")
cat("  - Low demand: 0 to", round(quantiles[1], 0), "players\n")
cat("  - Medium demand:", round(quantiles[1], 0), "to", round(quantiles[2], 0), "players\n")
cat("  - High demand:", round(quantiles[2], 0), "+ players\n")

### 4.6 Session Duration Analysis

Explore how session duration relates to player engagement and server resource consumption.

In [ ]:
# Create boxplot of session duration by time of day
ggplot(sessions_features, aes(x = time_of_day, y = duration_minutes, fill = time_of_day)) +
  geom_boxplot(outlier.alpha = 0.3, show.legend = FALSE) +
  scale_y_continuous(limits = c(0, quantile(sessions_features$duration_minutes, 0.95, na.rm = TRUE))) +
  scale_fill_brewer(palette = "Set2") +
  labs(
    title = "Session Duration by Time of Day",
    subtitle = "Boxplots show median, quartiles, and outliers (top 5% excluded for clarity)",
    x = "Time of Day",
    y = "Session Duration (minutes)"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 16, face = "bold"),
    plot.subtitle = element_text(size = 12, color = "gray40")
  )

# Summary statistics by time of day
duration_by_time <- sessions_features %>%
  group_by(time_of_day) %>%
  summarise(
    avg_duration = round(mean(duration_minutes, na.rm = TRUE), 2),
    median_duration = median(duration_minutes, na.rm = TRUE),
    max_duration = max(duration_minutes, na.rm = TRUE),
    session_count = n()
  ) %>%
  arrange(desc(avg_duration))

cat("\nSession Duration by Time of Day:\n")
print(duration_by_time)

cat("\nKey Insights:\n")
cat("  - Longest average sessions occur during", 
    as.character(duration_by_time$time_of_day[1]), "\n")
cat("  - This suggests higher player engagement during", 
    tolower(as.character(duration_by_time$time_of_day[1])), "hours\n")